# INFO370 Problem Set 6: Linear Regression
Group Member: Martin Zhang, 

In [4]:
import pandas as pd
import numpy as np

## 1 Data description

1.1 Load the data airbnb-seattle-listings-train.csv. Broadly describe the variables you see, their encoding, and discuss if these may be valuable in determining the price. For instance, you may want to tell that house_rules is text, and you may want to check if smoking allowed/not allowed is related to the price

In [7]:
data = pd.read_csv('./airbnb-seattle-listings-train.csv',sep='\t')

In [40]:
#changed pandas default col & rows display to view full columns
pd.options.display.max_columns = 106
pd.options.display.max_rows = 40
data

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2318,https://www.airbnb.com/rooms/2318,20190922030624,2019-09-22,Casa Madrona - Urban Oasis 1 block from the park!,"Gorgeous, architect remodeled, 1917 Dutch Colo...","Casa Madrona is a gorgeous, architect remodele...","Gorgeous, architect remodeled, 1917 Dutch Colo...",none,Madrona is a hidden gem of a neighborhood. It ...,"We adhere to a 10pm -9am quiet hour schedule, ...",NaN,Guests can access any part of the house.,We are a family who live next door and are ava...,NaN,NaN,NaN,https://a0.muscache.com/im/pictures/02973ad3-a...,NaN,2536,https://www.airbnb.com/users/show/2536,Megan,2008-08-26,"Seattle, Washington, United States",I welcome guests from all walks of life and ev...,within an hour,100%,NaN,t,https://a0.muscache.com/im/pictures/user/016a1...,https://a0.muscache.com/im/pictures/user/016a1...,Minor,2.0,2.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,f,"Seattle, WA, United States",Madrona,Madrona,Central Area,Seattle,WA,98122,Seattle,"Seattle, WA",US,United States,47.61082,-122.29082,t,House,Entire home/apt,9,2.5,4.0,4.0,Real Bed,"{Internet,Wifi,Kitchen,""""Free parking on premi...",NaN,$296.00,NaN,NaN,$500.00,$250.00,8,$25.00,30,1000,30,30,1000,1000,30.0,1000.0,5 days ago,t,25,55,84,84,2019-09-22,28,8,2008-09-15,2019-08-30,100.0,10.0,10.0,10.0,10.0,10.0,10.0,t,NaN,"{WASHINGTON,"""" Seattle"""","""" WA""""}",f,f,strict_14_with_grace_period,f,f,2,2,0,0,0.21
1,5682,https://www.airbnb.com/rooms/5682,20190922030624,2019-09-22,"Cozy Studio, min. to downtown -WiFi",The Cozy Studio is a perfect launchpad for you...,"Hello fellow travelers, Save some money and ha...",The Cozy Studio is a perfect launchpad for you...,none,NaN,My personal favorite places to dine are: • Ma...,Bus 120 is right outside and goes into downtow...,PRIVATE ENTRANCE WITH PRIVATE BATH Queen size ...,"I'm more than happy to meet with guests, but u...",**PLEASE READ** NO SMOKING NO VAPING NO MARIJU...,NaN,NaN,https://a0.muscache.com/im/pictures/5ec89f9d-8...,NaN,8993,https://www.airbnb.com/users/show/8993,Maddy,2009-03-03,"Seattle, Washington, United States",Hello my name is Maddy. I enjoy meeting and ho...,NaN,NaN,NaN,t,https://a0.muscache.com/im/users/8993/profile_...,https://a0.muscache.com/im/users/8993/profile_...,South Delridge,1.0,1.0,

In [130]:
#takes in a list of str and returns a list of integer
def priceConverter(li:list):
    res = []
    for i in li:
        #erase potential comma in the string
        i = float(i[1:].replace(',',''))
        res.append(int(i))
    return res

In [131]:
GatewoodPrice = priceConverter(list(data[data.neighbourhood == 'Gatewood'].price))
BelltownPrice = priceConverter(list(data[data.neighbourhood == 'Belltown'].price))
GateWood_avg = round(np.mean(GatewoodPrice),2)
Belltown_avg = round(np.mean(BelltownPrice),2)
print("location is apparently a signigicant factor. Belltown's average rent price is %s, while for Gatewood is %s." %(Belltown_avg,GateWood_avg)
)

location is apparently a signigicant factor. Belltown's average rent price is 222.16, while for Gatewood is 119.18.


1.2. Consider how will you handle missing data. For instance, 95% of the "square feet" observations are missing, 17% of "security deposit" observations are missing. You lose too many observations if you just ignore those.

If data is missing siginificantly(e.g. square feet), we choose to not consider the column and instead using some other measurment that can in some degree cover up the loss of this part of data. For example we can use property_type, bedroom, and bathroom.

1.3 Consider which variables you are going to use below. For all of these, create a summary table that contains relevant summary information. In particular pay attention to the missing values. Note that missings may not just be coded as such, they may also be empty strings and values like "N/A". You may return to this point repeatedly as you develop your model.

neighbourhood, property_type, room_type, accommodates, bathrooms, bedrooms, beds, bed_type, price, weekly_price, security_deposit, cleaning_fee, minimum_nights, review_scores_rating, review_scores_accuracy, review_scores_cleanliness, review_scores_checkin, review_scores_communication, review_scores_location, review_scores_value 

## 2 Model 

In [8]:
import re
def trimSpaces(n): 
    return re.sub('\$|,','',n)
usedData=["monthly_price", "is_business_travel_ready","experiences_offered", "reviews_per_month", "instant_bookable", "calculated_host_listings_count_private_rooms", "review_scores_cleanliness"]
usedData=list(map(trimSpaces,usedData))
listingsTrimmed=data[usedData]
listingsDropped=listingsTrimmed.dropna()
listingsDropped.loc[:,'monthly_price']=listingsDropped.loc[:,'monthly_price'].apply(trimSpaces).apply(lambda x:float(x))
formulaStr='monthly_price ~ is_business_travel_ready + experiences_offered + reviews_per_month + instant_bookable + calculated_host_listings_count_private_rooms + review_scores_cleanliness'
mod = smf.ols(formula=formulaStr, data=listingsDropped)
res = mod.fit()
#print(data.head(3))
print('I estimate the regression model to look something like',formulaStr)
print(res.summary())

//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


NameError: name 'smf' is not defined